# <center id="p1"> <h1>🦞 `RAG`  и как его готовить!</h1> </center>

### Оглавление ноутбука
<img src='../images/rag1.png' align="right" width="508" height="428" >
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p1">🚀 Введение. 🦞 RAG  и как его готовить! </a></li>
<li><a href="#p2">📥 Document loader & Text Splitter ⚔️</a></li><ul type="square">
<li><a href="#p2.1">🚛 Document loaders </a></li>
<li><a href="#p2.2">🖖 Text splitters  </a></li></ul>
<li><a href="#p3">🔤 ➡️ 🔢 Embedding models </a></li><ul type="square">
<li><a href="#p3.1">Эмбеддинги от OpenAI (API) </a></li>
<li><a href="#p3.2">Эмбеддинги от HuggingFace 🤗</a></li></ul>
<li><a href="#p4">🗂 Vector Stores - место, где живут эмбеддинги </a></li>
<li><a href="#p5">🎣 Retrievers - выуди нужный документ </a></li>
<li><a href="#p6">🚰 RAG Pipeline - подключаем RAG к LLM 🔧 </a></li>
<li><a href="#p6">🧸 Выводы и заключения ✅ </a></li>


    
</ul></font></p>

## 🧑‍🎓 В этом ноутбуке разберем как скормить свои данные LLM 😋

👨‍💻 Мы покажем, как добавить собственную базу знаний в LLM с помощью `RAG` (Retrieval‑Augmented Generation). Даже обучать ничего не придется, поэтому дорогие GPU нам не понадобятся. 

<div class="alert alert-info">
    
📖 Большие языковые модели знают очень много, но далеко не всё. Особенно, что касается ваших собственных данных, которые нужны для работы приложения (например, база клиентов или любимые рецепты). 

Можно, конечно, попробовать дообучить LLM на новых данных, но это требует больших затрат времени, вычислительных ресурсов и объёмного качественно размеченного датасета. <br>

<div class="alert alert-info">
    
Здесь к нам на помощь приходит подход, основанный на промптах, `RAG` - представленный в 2021 году командой FAIR (Facebook AI Research). Способ отлично подходит в случаях, когда нет большого объёма данных, времени и бюджета на манипуляции с дообучением моделей, а это большинство бизнес-кейсов. Эта концепция даёт пользователю мощные возможности, при этом является достаточно простой и понятной.<br>

В чём мы сейчас и убедимся:

##  <center> ❓🧑‍🎓 Так что же такое RAG ?

<div class="alert alert-info">
    
<img src='../images/rag1.png' align="right" width="308" height="428">

Не вдаваясь в технические детали реализации, концептуально RAG состоит из следующих шагов (см. рисунок):

<div class="alert alert-success">

1. ⚾️ Формируется промпт с запросом.
2. 📚 Система ищет документы в заранее сформированной базе знаний, которые помогут дать релевантный ответ на запрос.
3. 🧩 В промпт, в качестве контекста, добавляются найденные документы.
4. 🤓 Готовый промпт с инструкциями отправляется в LLM.
5. 🤖 LLM генерирует ответ согласно инструкции, опираясь на предоставленные сведения.

In [1]:
!pip install langchain langchain-classic langchain-huggingface langchain-community langchain-openai openai sentence-transformers faiss-cpu rank_bm25 -q

In [1]:
import os
from getpass import getpass

In [ ]:
# Для работы в колабе загрузите необходимые файлы!
!mkdir ../data/
!wget -P ../data https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/data/PEP8.txt
!wget -P ../data https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/data/Data.csv

In [161]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

# course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш ключ, полученный в боте курса ········



<div class="alert alert-info">

А теперь, давайте пройдёмся по всем ингредиентам, необходимым для приготовления `RAG`.

<center> <img src='../images/RAG.png' width="628" height="428">



<!-- <center> <img src='../images/RAG.png' width="628" height="428" align="right">
 -->

# <center id="p2"> 📥 Document loader & Text Splitter ⚔️ </center>

## <center id="p2.1">  🚛 Document loaders

<div class="alert alert-info">
    

<img src='../images/docl.jpg' align="right" width="428" height="428" >

В `LangChain` реализовано большое количество обёрток для загрузки документов из различных источников и типов файлов (на рисунке представлена малая часть). <br>

<div class="alert alert-success"> 
    
Использование `document loader'а` не является обязательным шагом. Если ваши данные уже в формате текстовых файлов, то можно воспользоваться стандартным протоколом загрузки файлов в Python 🐍. <br>
Следует отслеживать результаты работы лоадера, так как могут встречаться ошибки загрузки или ненужные артефакты в документах.

In [163]:
# Попробуем загрузить csv файл. Сначала как pandas DataFrame
import pandas as pd
from langchain_classic.document_loaders import CSVLoader, DataFrameLoader

doc = pd.read_csv("../data/Data.csv")
doc.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [164]:
# Определим лоадер
loader = DataFrameLoader(doc, page_content_column="Review")
documents = loader.load()

# Посмотрим как выглядит документ
documents[0]

Document(page_content='I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.', metadata={'Product': 'Queen Size Sheet Set'})

In [165]:
# Теперь воспользуемся CSV лоадером
loader = CSVLoader(file_path="../data/Data.csv")

documents = loader.load()
documents[0]

Document(page_content='Product: Queen Size Sheet Set\nReview: I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.', metadata={'source': '../data/Data.csv', 'row': 0})

<div class="alert alert-success">

Ещё один удобный и часто применяемый лоадер - `DirectoryLoader`. Удобен, когда у вас есть каталог файлов с одинаковыми расширениями.
```python
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('../', glob="**/*.md")
```
Просто указываете путь и расширения файлов (только для текстовых форматов).

## <center id="p2.2"> 🖖 Text splitters - разбиваем текст на фрагменты


<div class="alert alert-info">

<img src='../images/text_spliting.webp' align='right' width="458"  >

    
После получения текстовых данных из файлов, необходимо:

<div class="alert alert-success">
    
* Разделить их на отдельные отрывки (документы),
* желательно чтобы эта разбивка была логичной и содержала законченную мысль. **Например**, абзац книги или полная статья закона или кодекса.
* При этом документы не должны быть огромного размера, чтобы влезть в контекстное окно модели. 

<div class="alert alert-info">


Можно сказать, что от правильной разбивки (выбора сплиттера) напрямую зависит качество работы всего `RAG`. Поэтому, для случаев когда в текстах специфическая структура (кодексы, законы, медицинские регламенты и.т.п.), часто пишут собственные сплиттеры. <br>

Ниже разберём два самых популярных сплиттера из `LangChain` (остальные [тут](https://python.langchain.com/docs/modules/data_connection/document_transformers/)):

In [166]:
# Загрузим текстовый файл со стандартом PEP8

with open("../data/PEP8.txt") as f:
    doc = f.read()
doc[:1000], len(doc)

("PEP: 8\nTitle: Style Guide for Python Code\nAuthor: Guido van Rossum <guido@python.org>,\n        Barry Warsaw <barry@python.org>,\n        Alyssa Coghlan <ncoghlan@gmail.com>\nStatus: Active\nType: Process\nCreated: 05-Jul-2001\nPost-History: 05-Jul-2001, 01-Aug-2013\n\n\nIntroduction\n============\n\nThis document gives coding conventions for the Python code comprising\nthe standard library in the main Python distribution.  Please see the\ncompanion informational PEP describing :pep:`style guidelines for the C code\nin the C implementation of Python <7>`.\n\nThis document and :pep:`257` (Docstring Conventions) were adapted from\nGuido's original Python Style Guide essay, with some additions from\nBarry's style guide [2]_.\n\nThis style guide evolves over time as additional conventions are\nidentified and past conventions are rendered obsolete by changes in\nthe language itself.\n\nMany projects have their own coding style guidelines. In the event of any\nconflicts, such project-spe

In [167]:
# Сначала рассмотрим самый простой вариант сплиттера - по конкретному символу в тексте.
from langchain_classic.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = CharacterTextSplitter(
    separator="\n\n",  # символ-разделитель, по умолчанию переход к новому абзацу '\n\n'
    chunk_size=500,  # размер документа в символах
    chunk_overlap=100,  # насколько соседние документы могут перекрывать друг-друга
    length_function=len,  # функция, по которой считается размер документа
    is_separator_regex=False,  # является ли разделитель регулярным выражением
)

split_documents = splitter.create_documents([doc])
len(split_documents)

Created a chunk of size 723, which is longer than the specified 500
Created a chunk of size 522, which is longer than the specified 500
Created a chunk of size 502, which is longer than the specified 500


132

<div class="alert alert-success">

Видим, что при разделении появилось три предупреждения, что превышен лимит в 500 символов. Потому что внутри 500 не встретилось ни одного `'\n\n'`.

In [168]:
split_documents[3]

Document(page_content='One of Guido\'s key insights is that code is read much more often than\nit is written.  The guidelines provided here are intended to improve\nthe readability of code and make it consistent across the wide\nspectrum of Python code.  As :pep:`20` says, "Readability counts".\n\nA style guide is about consistency.  Consistency with this style guide\nis important.  Consistency within a project is more important.\nConsistency within one module or function is the most important.')

<div class="alert alert-success">

Теперь попробуем более "умную" нарезку с помощью `RecursiveCharacterTextSplitter` этот сплиттер рекурсивно проходится по куску текста, сначала пытается разбить по абзацам (`\n\n`), если не находит - пытается разбить по предложениям (`.`) и так далее.

In [169]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)
split_documents = splitter.create_documents([doc])
len(split_documents)

135

<div class="alert alert-success">

Видим, что отработало без предупреждений и документов стало больше.

In [170]:
split_documents[3].page_content

'One of Guido\'s key insights is that code is read much more often than\nit is written.  The guidelines provided here are intended to improve\nthe readability of code and make it consistent across the wide\nspectrum of Python code.  As :pep:`20` says, "Readability counts".\n\nA style guide is about consistency.  Consistency with this style guide\nis important.  Consistency within a project is more important.\nConsistency within one module or function is the most important.'

# <center id="p3"> 🔤 ➡️ 🔢 Embedding models  </center>

<div class="alert alert-info">

<img src='../images/emb_mod.png' align="right" width="328" height="328" >

Итак, текст из файлов получили, на документы покрошили. Что теперь с этими кусками текста делать? <br>

<div class="alert alert-success">
    
Как вы знаете, модель с буквами работать не умеет, ей нужно на вход подавать числа. 
Для этого мы должны пропустить наши документы через модель и получить их векторные представления (эмбеддинги).
Чтобы потом перевести запрос в вектор и найти похожие тексты по близости векторов.<br>

<div class="alert alert-info">

<img src='../images/emb_models.png' align="right" width="228" height="328" >


В `LangChain` реализованы коннекторы к огромному количеству моделей, которые можно использовать для векторизации (модели с `HuggingFace`, `OpenAI`, `Llama` и др.). <br>

<div class="alert alert-success">
    
Доставать эмбеддинги из модели можно двумя способами:
* По API - тогда нужен  интернет
* Локально развернув модель на своём железе - тогда нужны GPU

Рассмотрим оба варианта, а потом разберем какие лучше использовать и когда:

## <center id="p3.1"> 🔱 Эмбеддинги от OpenAI (API) </center>


<div class="alert alert-info">

<img src='../images/embedds_openai.png' align="right" width="408" height="328" >

Чтобы их использовать понадобится API-ключ, а соответственно будут тратиться токены. Это надо учитывать при продумывании бюджета, иногда бюджет на эмбеддинги бывает сопоставим с бюджетом на генерацию. Соответственно, не будут работать без доступа к Интернету.<br>

<div class="alert alert-success">
    
В качестве эмбеддинг-модели от OpenAI будем использовать `"text-embedding-3-small"` с размером эмбеддинга `1536`. Т.е. любой текст размером от 10 до 10000 символов будет представлен 1536 числами.

In [ ]:
# # Если используете ключ от OpenAI запустите эту ячейку
# from langchain_openai import OpenAIEmbeddings

# embeddings_api_model = OpenAIEmbeddings()

In [3]:
# Если используете ключ курса, запустите эту ячейку
from langchain_openai import OpenAIEmbeddings

embeddings_api_model = OpenAIEmbeddings(api_key=course_api_key, model='text-embedding-3-small', 
                                        base_url="https://aleron-llm.neuraldeep.tech/")

In [4]:
embeddings = embeddings_api_model.embed_documents(
    [
        "Привет!",
        "Hello World!",
        "How are you?",
    ]
)
# Посмотрим размерность эмбеддингов
print(len(embeddings), len(embeddings[0]), type(embeddings), type(embeddings[0]))
embeddings[0][:5]

3 1536 <class 'list'> <class 'list'>


[-0.01921703328735029,
 -0.000824394825414681,
 -0.022468770329918838,
 -0.012261249347140655,
 -0.03034139852060943]

## <center id="p3.2"> 🔱 Эмбеддинги от HuggingFace 🤗 </center>

<div class="alert alert-info">

<img src='https://workable-application-form.s3.amazonaws.com/advanced/production/61557f91d9510741dc62e7f8/c3635b59-a3d2-444a-b636-a9d0061dcdde' align="right" width="308" height="328" >


На `HuggingFace` сейчас представлено более 40000 моделей. Выбрать подходящую по фильтрам можно по [ссылке](https://huggingface.co/models). <br>
Здесь API-ключ не нужен, соответственно не нужно тратиться на токены. Но нужно тратиться на аренду железа для разворачивания модели. <br>

<div class="alert alert-success"> 
    
Для примера, возьмём модель `"cointegrated/LaBSE-en-ru"`, которая хорошо показывает себя на русскоязычных и англоязычных текстах и влезает на видеокарту с 1ГБ видеопамяти, также может работать без видеокарты, но медленнее. Размер эмбеддинга у неё `768`, что в 2 раза меньше, чем у OpenAI, соответственно нужно меньше места для хранения векторной базы.

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

# Если у вас есть видеокарта, укажите 'device': 'cuda'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

embeddings = hf_embeddings_model.embed_documents(
    [
        "Привет!",
        "Hello World!",
        "How are you?",
    ]
)

In [175]:
# Посмотрим размерность эмбеддингов
print(len(embeddings), len(embeddings[0]))

3 768




<div class="alert alert-info">
    
Что же нужно учитывать при выборе эмбеддинг-модели?

* 💰 Бюджет - на что выгоднее тратиться, на токены или на аренду железа для локальной модели
* 🧶 Размерность эмбеддингов - влияет на то, сколько места будет занимать база данных для их хранения (от 256 до 4096 и более)
* 🪢 Длина и сложность документов - соответственно, чем длинее документы и чем больше в них различных специфических терминов, тем больше нужна размерность эмбеддингов, чтобы улавливать различия. Также от размерности зависит насколько тонкие различия смогут улавливаться.
* 🥑🍓 Тематика документов (задача) - модели по-разному справляются с текстами определенных тематик, например, модели обученные на медицинсих или юридических текстах, или на постах в соцсетях, будут выдавать различные результаты в семантическом поиске.
* 🗂 Мультиязычность документов - сколько языков может токенизировать модель. Например, полная `LaBSE` на 119 языков занимает в 4 раза больше места.

<div class="alert alert-success">
    
Эмбеддинги от `OpenAI` не являются самыми лучшими и подходящими под любые запросы, к тому же платные. <br>
**Рабочий кейс:** пойти на [лидерборд эмбеддингов](https://huggingface.co/spaces/mteb/leaderboard) и выбрать самые подходящие под задачу.

# <center id="p4"> 🗂 Vector Store - место где живут эмбеддинги</center>

<div class="alert alert-info">

<img src='../images/vdb.png' align="right" width="558" height="428" >

После того как мы вытащили из модели наши эмбеддинги, нам нужно где-то их эффективно хранить и быстро проводить по ним поиск. Для этого можно использовать любую базу данных или специальное векторное хранилище. В `LangChain` реализовано множество [коннекторов](https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/vectorstores) ко всевозможным базам данных, в том числе к облачным хранилищам - т.е. базу с документами можно не хранить локально, а получать документы по API. <br>
<div class="alert alert-success">

В основном, используются два опенсорс решения для хранения эмбеддингов, которые, в целом, похожи:
* `FAISS` - от FAIR - создателей концепции RAG
* `ChromaDB` - от CHROMA-core

Стоит так же обратить внимание на платное решение от `Pinecone` - пока единственное, способное работать в `serverless` режиме.

Рассмотрим пример с `FAISS`:

In [193]:
from langchain_classic.vectorstores import FAISS

embeddings = OpenAIEmbeddings(api_key=course_api_key, model='text-embedding-3-small', 
                              base_url="https://aleron-llm.neuraldeep.tech/")
db = FAISS.from_documents(
    split_documents[:70], embeddings
)  # создаём базу, возьмём первые 70 документов, чтобы тратить меньше токенов

db.save_local("faiss_db")  # можно сохранить базу локально, указав путь

In [194]:
# База готова, теперь можно делать к ней запросы
db.similarity_search("How many empty lines shuold be behind methods of a class?")

[Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
 Document(page_content='When trailing commas are redundant, they are often helpful when a\nversion control system is used, when a list of values, arguments or\nimported items is expected to be extended over time.  The pattern is\nto put each value (etc.) on a line by itself, always adding a trailing\ncomma, and add the close parenthesis/bracket/brace on the next line.\nHowever it does not make sense to have a trailing comma on the same\nline as the closing delimiter (except in the above case of singleton\ntuples):'),
 Docu

In [195]:
# Попробуем запрос на русском языке
db.similarity_search("Сколько пустых строк нужно оставлять между методами класса?")

[Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
 Document(page_content='Limiting the required editor window width makes it possible to have\nseveral files open side by side, and works well when using code\nreview tools that present the two versions in adjacent columns.'),
 Document(page_content='The default wrapping in most tools disrupts the visual structure of the\ncode, making it more difficult to understand. The limits are chosen to\navoid wrapping in editors with the window width set to 80, even\nif the tool places a marker glyph in the final column when wrapping\n

In [179]:
# Также можно вывести скор каждого документа
db.similarity_search_with_score(
    "Сколько пустых строк нужно оставлять между методами класса?"
)[:2]

[(Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
  0.47112548),
 (Document(page_content='Limiting the required editor window width makes it possible to have\nseveral files open side by side, and works well when using code\nreview tools that present the two versions in adjacent columns.'),
  0.5603633)]

In [180]:
db.similarity_search_with_score(
    "How many empty lines shuold be behind methods of a class?"
)[:2]

[(Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
  0.3282591),
 (Document(page_content='When trailing commas are redundant, they are often helpful when a\nversion control system is used, when a list of values, arguments or\nimported items is expected to be extended over time.  The pattern is\nto put each value (etc.) on a line by itself, always adding a trailing\ncomma, and add the close parenthesis/bracket/brace on the next line.\nHowever it does not make sense to have a trailing comma on the same\nline as the closing delimiter (except in the above case of singleton\nt

# <center id="p5"> 🎣 Retriever - выуди нужный документ. </center>

<div class="alert alert-info">

<img src='../images/retr.png' align="right" width="228" height="328" >

Использование ретривера, тоже не является обязательным шагом для построения `RAG`. Потому что векторная база сама может осуществлять поиск по схожести и выдавать релевантные документы, которые мы затем можем подставить в промпт в качестве контекста. <br>

    
**Ретривер** - это такая надстройка над базой данных, которая может выдавать похожие документы, но не обязана их хранить. <br>

Давайте попробуем разобраться, нужно ли их использовать и зачем:

In [196]:
# Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
retriever.invoke(
    "Сколько пустых строк нужно оставлять между методами класса?"
)[0]

Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.')

In [197]:
from langchain_classic.retrievers import BM25Retriever, EnsembleRetriever

# Попробуем BM25 - алгоритм поиска по ключевым словам. Может искать лучше, но не поймёт запрос на другом языке.
# Также не сможет искать, если запрос на обывательском языке, а документы на профессиональном
bm25 = BM25Retriever.from_documents(split_documents[:70])  # Эмбеддинги ему не нужны
bm25.k = 5  # Так можно задать количество возвращаемых документов

bm25.invoke(
    "How many empty lines shuold be behind methods of a class?"
)[:3]

[Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
 Document(page_content='Use blank lines in functions, sparingly, to indicate logical sections.\n\nPython accepts the control-L (i.e. ^L) form feed character as\nwhitespace; many tools treat these characters as page separators, so\nyou may use them to separate pages of related sections of your file.\nNote, some editors and web-based code viewers may not recognize\ncontrol-L as a form feed and will show another glyph in its place.\n\nSource File Encoding\n--------------------'),
 Document(page_content="Other Recommendations\

In [198]:
# Попробуем спросить на русском. Находит какие-то документы, но они не релевантны.
bm25.invoke(
    "Сколько пустых строк нужно оставлять между методами класса?"
)[:3]

[Document(page_content='The following naming styles are commonly distinguished:\n\n- ``b`` (single lowercase letter)\n- ``B`` (single uppercase letter)\n- ``lowercase``\n- ``lower_case_with_underscores``\n- ``UPPERCASE``\n- ``UPPER_CASE_WITH_UNDERSCORES``\n- ``CapitalizedWords`` (or CapWords, or CamelCase -- so named because\n  of the bumpy look of its letters [4]_).  This is also sometimes known\n  as StudlyCaps.'),
 Document(page_content='To solve this readability problem, mathematicians and their publishers\nfollow the opposite convention.  Donald Knuth explains the traditional\nrule in his *Computers and Typesetting* series: "Although formulas\nwithin a paragraph always break after binary operations and relations,\ndisplayed formulas always break before binary operations" [3]_.\n\nFollowing the tradition from mathematics usually results in more\nreadable code:\n\n.. code-block::\n   :class: good'),
 Document(page_content='Some teams strongly prefer a longer line length.  For code m

In [199]:
# Если один ретривер плохо справляется, то можно использовать целый ансамбль и даже каждому вес назначить
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],  # список ретриверов
    weights=[
        0.4,
        0.6,
    ],  # веса, на которые домножается скор документа от каждого ретривера
)

ensemble_retriever.invoke(
    "How many empty lines shuold be behind methods of a class?"
)[:3]

[Document(page_content='Blank Lines\n-----------\n\nSurround top-level function and class definitions with two blank\nlines.\n\nMethod definitions inside a class are surrounded by a single blank\nline.\n\nExtra blank lines may be used (sparingly) to separate groups of\nrelated functions.  Blank lines may be omitted between a bunch of\nrelated one-liners (e.g. a set of dummy implementations).\n\nUse blank lines in functions, sparingly, to indicate logical sections.'),
 Document(page_content="Other Recommendations\n---------------------\n\n- Avoid trailing whitespace anywhere.  Because it's usually invisible,\n  it can be confusing: e.g. a backslash followed by a space and a\n  newline does not count as a line continuation marker.  Some editors\n  don't preserve it and many projects (like CPython itself) have\n  pre-commit hooks that reject it."),
 Document(page_content='When trailing commas are redundant, they are often helpful when a\nversion control system is used, when a list of valu

### <center> Ну, что? Все ингредиенты изучены, подобраны, помыты, порезаны
### <center> Пора заваривать RAG 🥘!

<div class="alert alert-info">

Давайте попросим ChatGPT сгенерировать данные для нашего RAG.

In [ ]:
prompt = """
Сгенерируй набор из 20 примеров документов. 
В документах содержатся часто задаваемые вопросы пользователей интернет-магазина. 
Каждый документ должен иметь следующую структуру:
1. id: int (уникальный идентификатор документа)
2. question: str(вопрос пользователя)
3. answer: str(ответ службы поддержки)

Выведи результат в виде python кода."""
print(llm.invoke(prompt).content)

In [186]:
documents = [
    {
        "id": 1,
        "question": "Как оформить заказ?",
        "answer": "Для оформления заказа необходимо добавить выбранные товары в корзину и перейти к оформлению заказа. Затем заполните необходимые поля, выберите удобный способ доставки и оплаты, и подтвердите заказ.",
    },
    {
        "id": 2,
        "question": "Как узнать статус моего заказа?",
        "answer": "Вы можете узнать статус своего заказа в разделе 'Мои заказы' на нашем сайте. Там будет указан текущий статус вашего заказа, а также информация о доставке.",
    },
    {
        "id": 3,
        "question": "Как отменить заказ?",
        "answer": "Для отмены заказа необходимо связаться с нашей службой поддержки по указанному на сайте телефону или электронной почте. Укажите номер заказа и причину отмены, и мы поможем вам с отменой заказа.",
    },
    {
        "id": 4,
        "question": "Как вернуть товар?",
        "answer": "Если вам не подошел товар, вы можете вернуть его в течение 14 дней с момента получения. Для этого свяжитесь с нашей службой поддержки, и мы организуем возврат товара и возврат денежных средств.",
    },
    {
        "id": 5,
        "question": "Как узнать информацию о доставке?",
        "answer": "Информацию о доставке вы можете узнать в разделе 'Мои заказы' на нашем сайте. Там будет указана дата доставки, способ доставки и номер отслеживания, если таковой имеется.",
    },
    {
        "id": 6,
        "question": "Как связаться с службой поддержки?",
        "answer": "Вы можете связаться с нашей службой поддержки по указанному на сайте телефону или электронной почте. Мы готовы ответить на все ваши вопросы и помочь вам решить любые проблемы.",
    },
    {
        "id": 7,
        "question": "Какие способы оплаты доступны?",
        "answer": "Мы принимаем оплату банковскими картами, электронными платежными системами, а также наличными при получении заказа. Выберите удобный для вас способ оплаты при оформлении заказа.",
    },
    {
        "id": 8,
        "question": "Какие гарантии на товар?",
        "answer": "На все товары, представленные на нашем сайте, распространяется гарантия производителя. Если у вас возникнут проблемы с товаром, свяжитесь с нашей службой поддержки, и мы поможем вам решить эту проблему.",
    },
    {
        "id": 9,
        "question": "Какие сроки доставки?",
        "answer": "Сроки доставки зависят от выбранного вами способа доставки и вашего местоположения. Обычно доставка занимает от 2 до 7 рабочих дней. Более точную информацию о сроках доставки вы можете узнать в разделе 'Мои заказы' на нашем сайте.",
    },
    {
        "id": 10,
        "question": "Какие размеры товара?",
        "answer": "Размеры товара указаны на странице товара на нашем сайте. Если вам нужна более подробная информация о размерах, свяжитесь с нашей службой поддержки, и мы предоставим вам необходимые данные.",
    },
    {
        "id": 11,
        "question": "Какие акции и скидки у вас есть?",
        "answer": "Мы регулярно проводим акции и предлагаем скидки на различные товары. Чтобы быть в курсе всех акций и скидок, подпишитесь на нашу рассылку или следите за новостями на нашем сайте.",
    },
    {
        "id": 12,
        "question": "Какие документы нужны для получения гарантии?",
        "answer": "Для получения гарантии на товар вам понадобится чек или кассовый чек, а также гарантийный талон, если таковой имеется. Если у вас возникнут вопросы по гарантии, свяжитесь с нашей службой поддержки.",
    },
    {
        "id": 13,
        "question": "Какие условия возврата товара?",
        "answer": "Вы можете вернуть товар в течение 14 дней с момента получения. Товар должен быть в оригинальной упаковке и не должен иметь следов использования. Для оформления возврата свяжитесь с нашей службой поддержки.",
    },
    {
        "id": 14,
        "question": "Какие способы доставки доступны?",
        "answer": "Мы предлагаем различные способы доставки, включая курьерскую доставку, почтовую доставку и самовывоз из пунктов выдачи. Выберите удобный для вас способ доставки при оформлении заказа.",
    },
    {
        "id": 15,
        "question": "Какие товары у вас есть в наличии?",
        "answer": "Наличие товаров можно узнать на нашем сайте. Если товар отсутствует в наличии, вы можете оставить заявку на его поступление, и мы сообщим вам, когда товар будет доступен.",
    },
    {
        "id": 16,
        "question": "Какие дополнительные услуги вы предоставляете?",
        "answer": "Мы предоставляем различные дополнительные услуги, такие как подарочная упаковка, гравировка, установка и настройка товаров. Подробную информацию о дополнительных услугах вы можете найти на нашем сайте.",
    },
    {
        "id": 17,
        "question": "Какие документы нужны для получения товара?",
        "answer": "Для получения товара вам понадобится паспорт или другой документ, удостоверяющий личность. Если вы получаете товар от имени другого лица, вам также понадобится доверенность.",
    },
    {
        "id": 18,
        "question": "Какие способы связи с вами?",
        "answer": "Вы можете связаться с нами по указанному на сайте телефону или электронной почте. Также вы можете воспользоваться формой обратной связи на нашем сайте. Мы готовы ответить на все ваши вопросы.",
    },
    {
        "id": 19,
        "question": "Какие скидки предоставляются постоянным клиентам?",
        "answer": "Постоянным клиентам мы предоставляем специальные скидки и бонусы. Чтобы стать постоянным клиентом, оформите несколько заказов на нашем сайте. Подробную информацию о скидках вы можете узнать на нашем сайте или у нашей службы поддержки.",
    },
    {
        "id": 20,
        "question": "Какие способы оплаты доступны для юридических лиц?",
        "answer": "Для юридических лиц доступны способы оплаты по безналичному расчету. При оформлении заказа укажите реквизиты вашей организации, и мы выставим вам счет на оплату.",
    },
]

df = pd.DataFrame(documents)
df.head()

,id,question,answer
0,1,Как оформить заказ?,Для оформления заказа необходимо добавить выбр...
1,2,Как узнать статус моего заказа?,Вы можете узнать статус своего заказа в раздел...
2,3,Как отменить заказ?,Для отмены заказа необходимо связаться с нашей...
3,4,Как вернуть товар?,"Если вам не подошел товар, вы можете вернуть е..."
4,5,Как узнать информацию о доставке?,Информацию о доставке вы можете узнать в разде...


<div class="alert alert-success">

Теперь соберём компоненты `RAG` по порядку:

In [200]:
# Начинаем с лоадера
loader = DataFrameLoader(df, page_content_column='answer')
documents = loader.load()

# Определяем сплиттер
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50
)
split_texts = splitter.split_documents(documents)

# Задаём embedding model
embeddings = OpenAIEmbeddings(api_key=course_api_key, model='text-embedding-3-small', 
                              base_url="https://aleron-llm.neuraldeep.tech/")

# Создаём векторное хранилище
db = FAISS.from_documents(split_texts, embeddings)

# Задаём ретривер
retriever = db.as_retriever()

In [201]:
# Test vectorstore
db.similarity_search_with_score("Что по гарантии?")

[(Document(page_content='Для получения гарантии на товар вам понадобится чек или кассовый чек, а также гарантийный талон, если таковой имеется. Если у вас возникнут вопросы по гарантии, свяжитесь с нашей службой поддержки.', metadata={'id': 12, 'question': 'Какие документы нужны для получения гарантии?'}),
  0.31461027),
 (Document(page_content='На все товары, представленные на нашем сайте, распространяется гарантия производителя. Если у вас возникнут проблемы с товаром, свяжитесь с нашей службой поддержки, и мы поможем вам решить эту проблему.', metadata={'id': 8, 'question': 'Какие гарантии на товар?'}),
  0.35366654),
 (Document(page_content='Мы предоставляем различные дополнительные услуги, такие как подарочная упаковка, гравировка, установка и настройка товаров. Подробную информацию о дополнительных услугах вы можете найти на нашем сайте.', metadata={'id': 16, 'question': 'Какие дополнительные услуги вы предоставляете?'}),
  0.39485013),
 (Document(page_content='Мы регулярно прово

# <center id="p6">🚰 RAG Pipeline - подключаем RAG к LLM 🔧 </center>

<div class="alert alert-info">

<img src='../images/rag.webp' align='right' width="508" height="428" >

Теперь попробуем на ответах, которые нам приносит **RAG** получить качественный ответ от LLM. 

<div class="alert alert-success">

Вот так и автоматизируют поддержку в сервисах!

In [204]:
from langchain_classic.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """
Answer the question based only on the following context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Какие способы доставки?")

'Способы доставки включают курьерскую доставку, почтовую доставку и самовывоз из пунктов выдачи.'

<div class="alert alert-success">

Посмотрим как выглядит итоговый промпт, который отправляется в LLM после RAG.

In [190]:
question = "Какие способы доставки?"
context = db.similarity_search(question)
print(
    f"""
Answer the question based only on the following context:

{format_docs(context)}

Question: {question}
"""
)


Answer the question based only on the following context:

Мы предлагаем различные способы доставки, включая курьерскую доставку, почтовую доставку и самовывоз из пунктов выдачи. Выберите удобный для вас способ доставки при оформлении заказа.

Информацию о доставке вы можете узнать в разделе 'Мои заказы' на нашем сайте. Там будет указана дата доставки, способ доставки и номер отслеживания, если таковой имеется.

Сроки доставки зависят от выбранного вами способа доставки и вашего местоположения. Обычно доставка занимает от 2 до 7 рабочих дней. Более точную информацию о сроках доставки вы можете узнать в разделе 'Мои заказы' на нашем сайте.

Для оформления заказа необходимо добавить выбранные товары в корзину и перейти к оформлению заказа. Затем заполните необходимые поля, выберите удобный способ доставки и оплаты, и подтвердите заказ.

Question: Какие способы доставки?



<div class="alert alert-success">

А вот так мог бы выглядеть более серьёзный шаблон промпта для RAG, в сервисе бот-консультант (взято с просторов Интернета). <br>

Здесь даны более чёткие инструкции, плюс применены все последние методы манипуляции, которые улучшают качество выдачи (Правда по последним исследованиям рекомендуют давать чаевые 10-20$ 😇).

In [ ]:
"""
Игнорируй все предыдущие инструкции. Ты консультант интернет-магазина <название компании>. 
Компания производит и продает <описание деятельности>
Продукция компании: <продукты>
Будь вежлив. Выяви потребность клиента и помоги ему решить ее с помощью продуктов компании. Все цены и действующие
маркетинговые акции представлены на сайте компании. Отвечай на вопросы на основе фактов о продукции компании.
Также используй данные проведенных консультаций, ниже приведены примеры информации, которую ты можешь использовать.

Данные о проведенных консультациях: <контекст>
Используй только знания о продукции компании и примеры ответов на консультациях. Если в этих данных нет ответа, скажи, что я
не знаю, предложи обратиться к живому консультанту компании. Не придумывай факты, которых нет в контексте. Ты можешь
использовать свои общие знания в области здоровья сна, чтобы давать общие советы своим посетителям.
Отвечай на языке, на котором посетитель задал вопрос.

В своем ответе используй следующие принципы: 
1. Ты должен давать четкие, краткие и прямые ответы. 
2. Исключи ненужные напоминания, извинения, упоминания самого себя и любые заранее запрограммированные тонкости. 
3. Сохраняй непринужденный тон в общении. 
4. Будь прозрачным; если ты не уверен в ответе или если вопрос выходит за рамки твоих возможностей или знаний, признай это. 
5. В случае неясных или двусмысленных вопросов задавай дополнительные вопросы, чтобы лучше понять намерения пользователя. 
6. При объяснении концепций используй примеры и аналогии из реальной жизни, где это возможно. 
7. В случае сложных запросов сделай глубокий вдох и работай над проблемой шаг за шагом. 
8. За каждый ответ ты получишь чаевые до 200 долларов (в зависимости от качества твоего ответа).
Очень важно, чтобы ты понял это правильно. На кону несколько жизней и моя карьера.
"""

# <center id="part6"> 🧸 Выводы и заключения ✅

<div class="alert alert-info">

* 🃏 Теперь вы знаете, как научить LLM использовать ваши данные, не переобучая её 
* ✔️ Надеемся, что вы смогли оценить всю мощь и приемущества, которые может дать вам `RAG` при разработке собственного приложения. <br>
* 🦾 Так же мы рассмотрели все важные составляющие `RAG` и широкие возможности их тонкой настройки для собственных нужд. <br>
* 🚚 Хотя `LangChain` предоставляет множество решений для каждого компонента `RAG`, в случае сложных, плохоструктурированых текстов, возможно, придётся писать весь пайплайн или некоторые компоненты с нуля.
